<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

### Library installation

This installs all required libraries for running the notebook.

In [ ]:
!pip install bt

The bt library is a flexible backtesting framework that bundles yfinance for data retrieval, pandas for data manipulation, and matplotlib for plotting. Installing bt pulls in all dependencies we need.

### Imports and setup

We use bt for building and running backtests with minimal boilerplate, and matplotlib.pyplot for visualizing equity curves and performance distributions.

In [ ]:
import bt
import matplotlib.pyplot as plt

### Download data and define strategy helpers

This downloads daily adjusted close prices for TLT (iShares 20+ Year Treasury Bond ETF) from Yahoo Finance over a 12-year period.

In [ ]:
data = bt.get("tlt", start="2010-01-01", end="2022-06-30")

Treasury bonds are a useful test asset because they exhibit seasonality patterns and tend to behave differently than equities. Using a long historical window gives us enough market cycles to assess whether any observed edge is real or just noise.

This function creates a bt Strategy object that selects all assets, applies target weights, and rebalances the portfolio on each trading day.

In [ ]:
def build_strategy(weights):
    return bt.Strategy(
        'wd', 
        [bt.algos.SelectAll(), 
         bt.algos.WeighTarget(weights), 
         bt.algos.Rebalance()]
    )

The bt framework uses an algorithm-based approach where each step in the trading logic is a composable building block. WeighTarget reads from a DataFrame of target weights, allowing us to define complex position-sizing rules outside the strategy itself.

This function wraps a strategy with price data, starting capital, and a commission model to create a runnable backtest.

In [ ]:
def build_backtest(strategy, df, initial_capital, commission_model):
    return bt.Backtest(
        strategy,
        df,
        initial_capital=initial_capital,
        commissions=commission_model,
    )

Separating the backtest configuration from the strategy definition makes it easy to test the same strategy under different assumptions. We can swap commission models or change initial capital without touching the trading logic.

This function extracts the day of month from the index and adds it as a column, which we need to implement our calendar-based trading rule.

In [ ]:
def add_dom(df):
    added = df.copy()
    added["day_of_month"] = df.index.day
    return added

Calendar-based features like day of month are commonly used to exploit institutional flow patterns. For example, pension funds and mutual funds often rebalance at month-end, creating predictable price pressure.

This function implements a tiered commission structure where larger trades pay higher fixed fees, simulating realistic brokerage costs.

In [ ]:
def commission_model(q, p):
    val = abs(q * p)
    if val > 2000:
        return 8.6
    if val > 1000:
        return 4.3
    if val > 100:
        return 1.5
    return 1.0

Transaction costs are one of the main reasons strategies that look good on paper fail in practice. Including commissions in our backtest helps us understand whether the edge is large enough to survive real-world trading friction.

This function creates target weights for a calendar-based strategy that is short during the first week, flat in the middle, and long during the last week of each month.

In [ ]:
def add_weights(df, symbol):
    strategy = df[[symbol]].copy()
    strategy.loc[:] = 0
    strategy.loc[df.day_of_month <= 7] = -1
    strategy.loc[df.day_of_month >= 23] = 1
    return strategy

This turn-of-the-month strategy attempts to capture a well-documented calendar anomaly in fixed income markets. By encoding our trading logic as a DataFrame of weights, we keep the signal generation separate from the execution machinery.

### Build and run the strategy backtest

This sets the starting capital for our backtest simulation.

In [ ]:
initial_capital = 10_000

This assembles all our components and runs the backtest, chaining together data preparation, weight calculation, strategy building, and execution.

In [ ]:
data_with_dom = add_dom(data)
weights = add_weights(data_with_dom, 'tlt')
strategy = build_strategy(weights)
backtest = build_backtest(strategy, data, initial_capital, commission_model)
first_res = bt.run(backtest)

The bt.run function simulates every trading day, tracking positions, cash, and performance metrics as it processes the price data. This is where all the heavy lifting happens under the hood.

This displays a comprehensive table of performance statistics including returns, risk metrics, and drawdown analysis.

In [ ]:
first_res.display()

The Sharpe ratio of 0.49 tells us the strategy earns about half a unit of return per unit of risk. The max drawdown of -19% shows the worst peak-to-trough decline we would have experienced. These numbers look reasonable, but we cannot yet tell if they are statistically significant or just luck.

This plots the equity curve showing how our portfolio value evolved over time.

In [ ]:
first_res.plot(figsize=(20, 10))

The equity curve visualizes the compounding of returns over the full backtest period. Flat or declining sections reveal drawdown periods, while steep upward slopes show when the strategy performed well.

### Validate strategy with permutation testing

This function randomly shuffles the order of daily prices while preserving the date index, breaking any real calendar relationship in the data.

In [ ]:
def shuffle_prices(df):
    shuffled = df.sample(frac=1)
    shuffled.index = df.index
    return shuffled

Permutation testing helps us answer a critical question: would our calendar-based strategy have worked just as well if there was no actual calendar pattern in prices? By shuffling prices, we create a null hypothesis where calendar effects cannot exist.

This stores the original strategy's Sharpe ratio and prepares to collect Sharpe ratios from 1000 randomized backtests.

In [ ]:
runs = 1000
initial_sharpe = first_res['wd'].daily_sharpe
sharpes = []

Running 1000 permutations gives us a distribution of outcomes under the null hypothesis. If our original Sharpe falls in the extreme tail of this distribution, we have evidence that the calendar pattern is real.

This loop runs the full backtest 1000 times on shuffled price data, collecting the Sharpe ratio from each run to build a null distribution.

In [ ]:
for run in range(0, runs):
    shuffled = shuffle_prices(data)
    shuffled_with_dom = add_dom(shuffled)
    weights = add_weights(shuffled_with_dom, 'tlt')
    strategy = build_strategy(weights)
    backtest = build_backtest(strategy, shuffled_with_dom, initial_capital, commission_model)
    res = bt.run(backtest)
    sharpe = res['wd'].daily_sharpe
    sharpes.append(sharpe)

Monte Carlo permutation testing is a non-parametric method that makes no assumptions about the distribution of returns. This makes it more robust than traditional statistical tests that assume normality, which rarely holds in financial data.

This plots a histogram of Sharpe ratios from the shuffled backtests with a vertical line showing where our actual strategy's Sharpe falls.

In [ ]:
dist = plt.hist(sharpes, bins=10)
plt.axvline(initial_sharpe, linestyle='dashed', linewidth=1)

The dashed line should fall far to the right of the histogram if our strategy has real predictive power. When the line sits inside the distribution, the performance could plausibly have come from random chance.

This computes the p-value by counting how many shuffled backtests produced a Sharpe ratio higher than our original strategy.

In [ ]:
N = sum(i > initial_sharpe for i in sharpes)
p_value = N / runs

This displays the p-value, which represents the probability of seeing our strategy's performance by random chance alone.

In [ ]:
p_value

A p-value of 0.001 means only 1 out of 1000 random shuffles beat our strategy. This is strong evidence that our calendar-based edge is real and not a statistical fluke. In practice, quants typically require p-values below 0.05 or 0.01 before considering a strategy worth trading.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.